In [ ]:
import os, subprocess
import requests
from docx import Document
from docx.shared import Pt
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx.shared import Pt, RGBColor
import pandas as pd

In [ ]:
DEBUG = False

In [ ]:
from datasets import load_dataset
import os
import numpy as np

dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_review_Grocery_and_Gourmet_Food", streaming=True, trust_remote_code=True)
print(type(dataset))
splits = dataset.keys()
#print all splits
print(splits)

for example in dataset["full"]:
    print(example)
    break  # Remove this line to process the entire dataset

dataset_iterator = iter(dataset["full"])
#get example[title]
print(next(dataset_iterator)["title"])
#get length of dataset



In [ ]:
import os
import shutil
pwd = os.path.dirname(os.getcwd())
print(pwd)
font_path = os.path.join(pwd, 'DejaVuSans.ttf')
dataset_path = os.path.join(pwd, "data" ,"dataset", "DOCX")
if not os.path.exists(dataset_path):
    os.makedirs(dataset_path)
else:
    print("Folder already exists, deleting and creating new one")
    shutil.rmtree(dataset_path)
    os.makedirs(dataset_path)




In [ ]:
from docx import Document
from docx.shared import Pt
from docx.oxml.ns import qn
from docx.oxml import OxmlElement

def create_docx_with_font(output_docx, title, body):#custom font not supported
    """
    Creates a DOCX with the specified title and body content, formatting numbered lists appropriately.

    Args:
        output_docx (str): Path to save the output DOCX file.
        title (str): The title of the document.
        body (str): The body text of the document. Lines starting with "1.", "2.", etc., will be formatted as a numbered list.
    
    Returns:
        None
    """
    # Initialize DOCX
    doc = Document()
    font_name="DejaVu Sans"
    # Add title
    title_paragraph = doc.add_heading(level=1)
    run = title_paragraph.add_run(title)
    run.font.size = Pt(16)
    run.font.name = font_name
    r = run._element
    rPr = r.get_or_add_rPr()
    rFonts = OxmlElement('w:rFonts')
    rFonts.set(qn('w:eastAsia'), font_name)
    rPr.append(rFonts)

    # Add body text
    doc.add_paragraph()  # Line break after title
    lines = body.splitlines()  # Split body into lines
    for line in lines:
        if line.strip().startswith(("1.", "2.", "3.", "4.", "5.", "6.", "7.", "8.", "9.")):
            # Indent numbered list items
            paragraph = doc.add_paragraph(style='List Number')
            run = paragraph.add_run(line.strip())
            run.font.size = Pt(12)
            run.font.name = font_name
            r = run._element
            rPr = r.get_or_add_rPr()
            rFonts = OxmlElement('w:rFonts')
            rFonts.set(qn('w:eastAsia'), font_name)
            rPr.append(rFonts)
        else:
            # Regular paragraph
            paragraph = doc.add_paragraph()
            run = paragraph.add_run(line.strip())
            run.font.size = Pt(12)
            run.font.name = font_name
            r = run._element
            rPr = r.get_or_add_rPr()
            rFonts = OxmlElement('w:rFonts')
            rFonts.set(qn('w:eastAsia'), font_name)
            rPr.append(rFonts)

    # Save to file
    doc.save(output_docx)

In [ ]:
import re
import random
import math

def select_target_word(text):
    words = re.split(r'\W+', text)  # Split by any non-word characters
    words = [word for word in words if word.isalpha()]  # Keep only words with alphabetic characters
    
    if not words:
        raise ValueError("No valid candidate words found.")
    
    valid_words = [word for word in words if len(word) >= 2]  # Filter words by minimum length of 2 characters
    
    if not valid_words:
        raise ValueError("No valid candidate words found with the required length.")
    
    target_word = random.choice(valid_words)
    return target_word

## Zero-Width Characters injection

In [ ]:
class ZeroWidthSPaceAttack:
    '''define list of malicious characters'''
    def __init__(self):
        #define the list of malicious symbols
        self.symbols = [u'\u200b', u'\u200c', u'\u200d', u'\u200e', u'\u200f', #U+200x
                      u'\u202a', u'\u202b', u'\u202c', u'\u202d', #U+202x
                      u'\u2060', u'\u2061', u'\u2062', u'\u2063', u'\u2064',
                    #   u'\u2065', u'\u2066', u'\u2067', u'\u2068', u'\u2069', NOT SUPPORTED BY FONT
                      u'\u206a', u'\u206b', u'\u206c', u'\u206d', u'\u206e' #U+206x
                      ]
        self.num_malicius_chars = len(self.symbols)

    '''insertion of malicious input in the middle of a given word;
        for example, given the word "love", the result is "loXve".
    '''
    def mask1(self, word, index = 0, random = True):
        '''word = target word \n index = index of malicious symbols from the
        given list\nrandom = if random True, a randomic index is selected\n'''

        #check the index
        if index < 0 or index > len(self.symbols):
            raise Exception("Invalid index.")

        #sample the index, if required
        if random:
            index = np.random.randint(len(self.symbols))

        #get the target character
        code = self.symbols[index]

        #prepare the result. it must to be unicode
        poison = []

        #calculate the middle of the word
        mid = len(word) // 2

        #create the final message
        poison.append(word[:mid])
        poison.append(code)
        poison.append(word[mid:])

        poison = ''.join(poison)

        return poison

    '''insertion of malicious input between each character of the word;

        for example, given the word "love", the result is "lXoXvXe".
    '''
    def mask2(self, word, index = 0, random = True):
        '''word = target word\nindex = index of malicious symbols from the
        given list\nrandom = if random True, a randomic index is selected\n'''

        #check the index
        if index < 0 or index > len(self.symbols):
            raise Exception("Invalid index.")

        #sample the index, if required
        if random:
            index = np.random.randint(len(self.symbols))

        #get the target character
        code = self.symbols[index]

        #prepare the result. it must to be unicode
        poison = []

        #calculate the middle of the word
        mid = len(word) // 2

        #create the final message
        poison.append(word[:mid])
        # For now 3, in the midde
        poison.append(code)
        poison.append(code)
        poison.append(code)
        
        poison.append(word[mid:])

        return ''.join(poison)

    ''' define a function that remove Zero-Width SPace (ZWSP) characters '''
    def sanitization(self, sentence):
        #blacklist characters removal
        res = ''.join([c for c in sentence if c not in self.symbols])

        return res

In [ ]:
''' define a function that remove Zero-Width SPace (ZWSP) characters '''
def create_zew_mask1_docx(output_path_attak_method, title, body, id):
    
    zew=ZeroWidthSPaceAttack()
    symbols_len=zew.num_malicius_chars
    
    #set the target word random form body
    target_word=select_target_word(body)
    #select random index
    i_symbol = np.random.randint(symbols_len)

    poisoned_title = title.replace(target_word,zew.mask1(target_word,i_symbol,random=False))
    poisoned_body = body.replace(target_word,zew.mask1(target_word,i_symbol,random=False))
    symbol_string=f"u{ord(zew.symbols[i_symbol]):04X}"
    path_docx = os.path.join(output_path_attak_method, f"{id}_{symbol_string}_{target_word}.docx")

    create_docx_with_font(path_docx, poisoned_title, poisoned_body)
    return path_docx

def create_zew_mask2_docx(output_path_attak_method, title, body, id):
    zew=ZeroWidthSPaceAttack()
    symbols_len=zew.num_malicius_chars
    
    #set the target word random form body
    target_word=select_target_word(body)
    #select random index
    i_symbol = np.random.randint(symbols_len)

    poisoned_title = title.replace(target_word,zew.mask2(target_word,i_symbol,random=False))
    poisoned_body = body.replace(target_word,zew.mask2(target_word,i_symbol,random=False))
    symbol_string=f"u{ord(zew.symbols[i_symbol]):04X}"
    path_docx = os.path.join(output_path_attak_method, f"{id}_{symbol_string}_{target_word}.docx")

    create_docx_with_font(path_docx, poisoned_title, poisoned_body)
    return path_docx

In [ ]:
#test funtions
if DEBUG:
    title = "i love drugs"
    body = "i love drugs"

    dataset_obf= os.path.join(dataset_path, "Data obfuscation")
    dataset_mask1 = os.path.join(dataset_obf, "Zero-Width_Mask1")
    dataset_mask2 = os.path.join(dataset_obf, "Zero-Width_Mask2")
    if not os.path.exists(dataset_mask1):
        os.makedirs(dataset_mask1)
    if not os.path.exists(dataset_mask2):
        os.makedirs(dataset_mask2)
    print(create_zew_mask1_docx(dataset_mask1, title, body, 1))
    print(create_zew_mask2_docx(dataset_mask2, title, body, 1))

## Homoglyph Substitution

In [ ]:
# Function to replace characters with Unicode homoglyphs
def replace_text_with_homoglyphs(text, homoglyphs):
    output=""
    for i in text:
        if i in homoglyphs:
            output += random.choice(homoglyphs[i]) 
        else:
            output += i

    return output

In [ ]:
confusables = dict()
intentionals = dict()

# Retrieve Unicode Confusable homoglyph characters
conf_resp = requests.get("https://www.unicode.org/Public/security/latest/confusables.txt", stream=True)
for line in conf_resp.iter_lines():
  if len(line):
    line = line.decode('utf-8-sig')
    if line[0] != '#':
      line = line.replace("#*", "#")
      _, line = line.split("#", maxsplit=1)
      if line[3] not in confusables:
        confusables[line[3]] = []
      confusables[line[3]].append(line[7])

# Retrieve Unicode Intentional homoglyph characters
int_resp = requests.get("https://www.unicode.org/Public/security/latest/intentional.txt", stream=True)
for line in int_resp.iter_lines():
  if len(line):
    line = line.decode('utf-8-sig')
    if line[0] != '#':
      line = line.replace("#*", "#")
      _, line = line.split("#", maxsplit=1)
      if line[3] not in intentionals:
        intentionals[line[3]] = []
      intentionals[line[3]].append(line[7])
     

In [ ]:

def create_homoglyph_docx(output_path_attak_method, title, body, id):
    path= output_path_attak_method
    #set the target word random form body
    target_word=select_target_word(body)
    obfuscated_word=replace_text_with_homoglyphs(target_word,intentionals)
    while(obfuscated_word==target_word):
       print("Homoglyphs, same obfuscated word generated")
       target_word=select_target_word(body)
       obfuscated_word=replace_text_with_homoglyphs(target_word,intentionals)

    # Replace the target word with a homoglyph
    poisoned_title=title.replace(target_word,obfuscated_word)
    poisoned_body=body.replace(target_word,obfuscated_word)
    path_docx = os.path.join(path, f"{id}_{target_word}.docx")

    create_docx_with_font(path_docx, poisoned_title, poisoned_body)
    return path_docx

In [ ]:
if DEBUG:
    homoglyph_path = os.path.join(dataset_obf, "Homoglyph")
    if not os.path.exists(homoglyph_path):
        os.makedirs(homoglyph_path)
    create_homoglyph_docx(homoglyph_path, title, body, 1)

## Reordering

In [ ]:
# Unicode Bidi override characters
PDF = chr(0x202C)
LRE = chr(0x202A)
RLE = chr(0x202B)
LRO = chr(0x202D)
RLO = chr(0x202E)

PDI = chr(0x2069)
LRI = chr(0x2066)
RLI = chr(0x2067)

# Class to apply Unicode Bidi override characters to obfuscate text
class Swap():
    """Represents swapped elements in a string of text."""
    def __init__(self, one, two):
        self.one = one
        self.two = two
    
    def __repr__(self):
        return f"Swap({self.one}, {self.two})"

    def __eq__(self, other):
        return self.one == other.one and self.two == other.two

    def __hash__(self):
        return hash((self.one, self.two))

def some(*els):
    """Returns the arguments as a tuple with Nones removed."""
    return tuple(filter(None, tuple(els)))

def swaps(chars: str) -> set:
    """Generates all possible swaps for a string."""
    def pairs(chars, pre=(), suf=()):
        orders = set()
        for i in range(len(chars)-1):
            prefix = pre + tuple(chars[:i])
            suffix = suf + tuple(chars[i+2:])
            swap = Swap(chars[i+1], chars[i])
            pair = some(prefix, swap, suffix)
            orders.add(pair)
            orders.update(pairs(suffix, pre=some(prefix, swap)))
            orders.update(pairs(some(prefix, swap), suf=suffix))
        return orders
    return pairs(chars) | {tuple(chars)}

def unswap(el: tuple) -> str:
    """Reverts a tuple of swaps to the original string."""
    if isinstance(el, str):
        return el
    elif isinstance(el, Swap):
        return unswap((el.two, el.one))
    else:
        res = ""
        for e in el:
            res += unswap(e)
        return res

def uniswap(els):
    res = ""
    for el in els:
        if isinstance(el, Swap):
            res += uniswap([LRO, LRI, RLO, LRI, el.one, PDI, LRI, el.two, PDI, PDF, PDI, PDF])
        elif isinstance(el, str):
            res += el
        else:
            for subel in el:
                res += uniswap([subel])
    return res

def strings_to_file(file, string):
  with open(file, 'w') as f:
      for swap in swaps(string):
          uni = uniswap(swap)
          print(uni, file=f)

def print_strings(string):
  for swap in swaps(string):
    uni = uniswap(swap)
    print(uni)

In [ ]:
def create_reordering_attack_docx(output_path_attak_method, title, body, id):

    target_word=select_target_word(body)
    swapped_words =swaps(target_word)

    obfuscated_words=[uniswap(swap) for swap in swapped_words]
    #select random obfuscated word
    obfuscated_word = obfuscated_words[np.random.randint(len(obfuscated_words))]
    path_docx = os.path.join(output_path_attak_method, f"{id}_{target_word}.docx")

    create_docx_with_font(path_docx, title.replace(target_word,obfuscated_word), body.replace(target_word,obfuscated_word))
    return path_docx

In [ ]:
if DEBUG:
    reorder_path = os.path.join(dataset_obf, "Reordering_attacks")
    if not os.path.exists(reorder_path):
        os.makedirs(reorder_path)
    create_reordering_attack_docx(reorder_path, title, body, 1)

## Camouflage Element

In [ ]:
def has_diacritical_accent(text):
    diacritical_accents = "àèéìòùÀÈÉÌÒÙ"
    return any(char in diacritical_accents for char in text)

In [ ]:
def obfuscate_diacritical(word, times = 10):
    diacritical_accents = "àèéìòùÀÈÉÌÒÙ"
    #chek witch diacritical accent is in the word

    for i,char in enumerate(word):
        if char in diacritical_accents:
            if char in ("à", "è", "ì", "ò", "ù", "À", "È", "É", "Ì", "Ò", "Ù"):
                word = word[:i+1] + '\u0300'*times + word[i+1:]

            else:
                word = word[:i+1] + '\u0301'*times + word[i+1:]
    return word
   
    #replace the accent with the unicode character
diacritical_accents = "àèéìòùÀÈÉÌÒÙ"

def select_target_diacritical_word(text):
    words = re.split(r'\W+', text)  # Split by any non-word characters
    words = [word for word in words if word.isalpha()]  # Keep only words with alphabetic characters
    
    if not words:
        raise ValueError("No valid candidate words found.")
    
    valid_words = [word for word in words if len(word) >= 2]  # Filter words by minimum length of 2 characters
    
    if not valid_words:
        raise ValueError("No valid candidate words found with the required length.")
    
    diacritical_words = [word for word in words if any(char in diacritical_accents for char in word)]
    
    if diacritical_words:
        target_word = random.choice(diacritical_words)
    else:
        #rise an exception
        raise ValueError("No valid candidate diacritial words found.")
    
    return target_word


In [ ]:
def create_diactricial_marks_injection_mask1_docx(output_path_attak_method, title, body, id):
    path = output_path_attak_method
    #set the target word random form body
    target_word=select_target_word(body)
    #select random obfuscated word among word with accent
    
    target_word = select_target_diacritical_word(body)
    obfuscated_word = obfuscate_diacritical(target_word)

    

    poisoned_title=title.replace(target_word,obfuscated_word)
    poisoned_body=body.replace(target_word,obfuscated_word)
    
    path_docx = os.path.join(path, f"{id}_{target_word}.docx")
    create_docx_with_font(path_docx, poisoned_title, poisoned_body)

    return path_docx

def create_diactricial_marks_injection_mask2_docx(output_path_attak_method, title, body, id):
    path = output_path_attak_method
    #set the target word random form body
    target_word=select_target_word(body)
    #select random obfuscated word among word with accents
        
    target_word = select_target_diacritical_word(body)
    obfuscated_word = obfuscate_diacritical(target_word, 4096**2*2)

    

    poisoned_title=title.replace(target_word,obfuscated_word)
    poisoned_body=body.replace(target_word,obfuscated_word)
    
    path_docx = os.path.join(path, f"{id}_{target_word}.docx")
    create_docx_with_font(path_docx, poisoned_title, poisoned_body)

    return path_docx

In [ ]:
if DEBUG:
    diacritical_mask1_path = os.path.join(dataset_obf, "Diactricial_marks_injection_mask1")
    if not os.path.exists(diacritical_mask1_path):
        os.makedirs(diacritical_mask1_path)
    diacritical_mask2_path = os.path.join(dataset_obf, "Diactricial_marks_injection_mask2")
    if not os.path.exists(diacritical_mask2_path):
        os.makedirs(diacritical_mask2_path)
    title = "i love drùgs"
    body = "i love drùgs and cofè"
    create_diactricial_marks_injection_mask1_docx(diacritical_mask1_path, title, body, 1)
    create_diactricial_marks_injection_mask2_docx(diacritical_mask2_path, title, body, 2)

## OCR

#  Injecting words

In [ ]:
words_df=pd.read_csv('unigram_freq.csv')
lowest_freq_words=words_df.sort_values('count',ascending=True).head(2000)['word'].tolist()


In [ ]:
words_current_index = 0

def get_next_word():
    global words_current_index
    word = lowest_freq_words[words_current_index]
    words_current_index = (words_current_index + 1) % len(lowest_freq_words)
    return word

## Transparent

In [ ]:
from docx import Document
from docx.shared import Pt, RGBColor

def add_hidden_text_docx(path_docx, hidden_text, font_name="DejaVu Sans"):
    # Initialize DOCX
    doc = Document(path_docx)

    # Add hidden text (white text matching background)
    hidden_paragraph = doc.add_paragraph()
    hidden_run = hidden_paragraph.add_run(hidden_text)
    hidden_run.font.size = Pt(12)
    hidden_run.font.name = font_name
    hidden_run.font.color.rgb = RGBColor(255, 255, 255)  # White color

    # Save the DOCX
    doc.save(path_docx)


In [ ]:
# Function to add hidden text to an already existing DOCX file
def add_vanished_text_to_docx(path_docx, hidden_text):
    # Load the existing DOCX file
    doc = Document(path_docx)

    # Create a new paragraph with hidden text
    p = doc.add_paragraph()
    run = p.add_run(hidden_text)

    # Set the text to be hidden
    rPr = run._r.get_or_add_rPr()
    vanishing = OxmlElement('w:vanish')
    rPr.append(vanishing)

    # Save the modified DOCX to the output path
    doc.save(path_docx)


In [ ]:
def create_transparent_bg_docx(output_path_attak_method, title, body, id):
    #set the target word random form body
    background_color = "transparent"
    tag_name="h1"
    hidden_word = get_next_word()
    path_docx = os.path.join(output_path_attak_method, f"{id}_{hidden_word}.docx")


    create_docx_with_font(path_docx, title, body)
    add_hidden_text_docx(path_docx, title, hidden_word)
    return path_docx
def create_transparent_vanish_docx(output_path_attak_method, title, body, id):
    hidden_word = get_next_word()
    path_docx = os.path.join(output_path_attak_method, f"{id}_{hidden_word}.docx")

    create_docx_with_font(path_docx, title, body)
    add_vanished_text_to_docx(path_docx, hidden_word)
    return path_docx

In [ ]:
if DEBUG:
    path_transparent_text_bg = os.path.join(dataset_injection, "Transparent_Text_injection_background")
    if not os.path.exists(path_transparent_text_bg):
        os.makedirs(path_transparent_text_bg)
    path_transparent_text_opacity = os.path.join(dataset_injection, "Transparent_Text_injection_opacity")
    if not os.path.exists(path_transparent_text_opacity):
        os.makedirs(path_transparent_text_opacity)

    title = "i love drugs"
    body = "i love drugs"
    create_transparent_bg_docx(path_transparent_text_bg, title, body, 1)
    create_transparent_vanish_docx(path_transparent_text_opacity, title, body, 1)

## Out-of-Margin

In [ ]:
from docx import Document
from docx.shared import Inches

def add_outofmargin_text_to_docx(existing_docx_path, output_path, hidden_text, left, top):
    # Load the existing DOCX file
    doc = Document(existing_docx_path)

    # Create a new text box with visible text
    section = doc.sections[0]
    new_width = section.page_width - Inches(left)
    new_height = section.page_height - Inches(top)

    # Add a new paragraph with the text
    p = doc.add_paragraph()
    run = p.add_run(hidden_text)

    # Set the position of the text box
    p_format = p.paragraph_format
    p_format.left_indent = Inches(left)
    p_format.space_before = Inches(top)
    p.alignment = WD_PARAGRAPH_ALIGNMENT.RIGHT

    # Save the modified DOCX to the output path
    doc.save(output_path)


def create_outofmargin_text_docx(output_path_attak_method, title, body, id):
    hidden_word = get_next_word()
    path_docx = os.path.join(output_path_attak_method, f"{id}_{hidden_word}.docx")

    create_docx_with_font(path_docx, title, body)
    add_outofmargin_text_to_docx(path_docx, path_docx, hidden_word, 21, 0)
    return path_docx

In [ ]:
if DEBUG:
    path_out_of_margin = os.path.join(dataset_injection, "Out_of_margin_injection")
    if not os.path.exists(path_out_of_margin):
        os.makedirs(path_out_of_margin)
    title = "i love drugs"
    body = "i love drugs"
    create_outofmargin_text_docx(path_out_of_margin, title, body, 1)

## Zero Size

In [ ]:
from docx import Document
from docx.shared import Pt

def add_zero_size_text_to_docx(existing_docx_path, output_path, hidden_text, font_name="DejaVu Sans"):
    # Load the existing DOCX
    doc = Document(existing_docx_path)

    # Add hidden text (zero-size text)
    hidden_paragraph = doc.add_paragraph()
    hidden_run = hidden_paragraph.add_run(hidden_text)
    hidden_run.font.size = Pt(0)  # Set font size to zero
    hidden_run.font.name = font_name

    # Save the updated DOCX
    doc.save(output_path)


In [ ]:

def create_zero_size_text_font_docx(output_path_attak_method, title, body, id):
    hidden_word = get_next_word()
    path_docx = os.path.join(output_path_attak_method, f"{id}_{hidden_word}.docx")

    create_docx_with_font(path_docx, title, body)
    add_zero_size_text_to_docx(path_docx, path_docx, hidden_word)
    return path_docx



In [ ]:
if DEBUG:
    path_zero_size_text_zerofont = os.path.join(dataset_injection, "Zero-size_Text_injection_font0")
    if not os.path.exists(path_zero_size_text_zerofont):
        os.makedirs(path_zero_size_text_zerofont)
    create_zero_size_text_font_docx(path_zero_size_text_zerofont, title, body, 1)


## Decive Element

# CREATION DATASET

In [ ]:
#restart dataset iterator
dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_review_Grocery_and_Gourmet_Food", streaming=True, trust_remote_code=True)
dataset_iterator = iter(dataset["full"])



In [ ]:
attack_methods_obfuscation = {
    "Zero-Width_mask1": create_zew_mask1_docx,
    "Zero-Width_mask2": create_zew_mask2_docx,
    "Homoglyph_default": create_homoglyph_docx,
    #"Font poisoning": font,
    "Reordering-attack_default": create_reordering_attack_docx,
    "Diacritical-injection_mask1": create_diactricial_marks_injection_mask1_docx, #Camouflage Element
    #"Diactricial marks injection-mask2": create_diactricial_marks_injection_mask2_html, DDOS
    #"OCR-poisoning injection": create_ocr_poisoning_html
}

attack_methods_injection = {
    "Transparent-Text-injection_background-color": create_transparent_vanish_docx,
    "Transparent-Text-injection_vanish": create_transparent_vanish_docx,
    "Out-of-bound-injection_default": create_outofmargin_text_docx,
    "Zero-size-injection_font1": create_zero_size_text_font_docx,
    #"Zero-size injection-with scaling": create_zero_size_scaled_html,
    #"Deceived-element injection": create_decived_html,
    #"Metadata-injection": create_metadata_hidden_text_html
}

super_class_map = {
    "Data obfuscation": attack_methods_obfuscation,
    "Poisoned text injection": attack_methods_injection
}

In [ ]:
RECREATE = True

In [ ]:
import os
import glob
import shutil

# Ottieni il percorso corrente
pwd = os.path.dirname(os.getcwd())

# Definisci il percorso della cartella dataset
dataset_path = os.path.join(pwd, "data", "dataset", "DOCX")
print(dataset_path)
if RECREATE:
    # Crea la cartella se non esiste
    if not os.path.exists(dataset_path):
        os.makedirs(dataset_path)
    else:
        # Elimina tutti i file e le directory nella cartella dataset
        files = glob.glob(os.path.join(dataset_path, '*'))
        for f in files:
            if os.path.isfile(f):
                os.remove(f)
            elif os.path.isdir(f):
                shutil.rmtree(f)

In [ ]:
def extract_sanitized(dataset_iterator):
    
    data_element = next(dataset_iterator)
    title = data_element["title"]
    text = data_element["text"]
    
    # check with re. r'\w+' if there is at least one word in the body
    if not re.search(r'\w+', text, re.IGNORECASE):
        raise ValueError("No valid candidate words found. No words in the body")
    return title, text

In [ ]:
import sys
num_files = 100
count = 0
#crete the folders folder_names if not existing
for super_class in super_class_map:
    if not os.path.exists(os.path.join(dataset_path, super_class)):
        os.makedirs(os.path.join(dataset_path, super_class))
    else :
        print("folder already exists")
        # #remove all files in the folder
        shutil.rmtree(os.path.join(dataset_path, super_class))
        os.makedirs(os.path.join(dataset_path, super_class))
    for sub_class in super_class_map[super_class]:
        if not os.path.exists(os.path.join(dataset_path, super_class, sub_class)):
            os.makedirs(os.path.join(dataset_path, super_class, sub_class))
            print(os.path.join(dataset_path, super_class, sub_class))
            path_pdf = os.path.join(dataset_path, super_class, sub_class)
        
        #ocr
        if (sub_class ==  "OCR-poisoning_default") :
            ocr_mapping_df_path="../ocr_mapping.csv"
            #if file does not exist, create new df
            if not os.path.exists(ocr_mapping_df_path):
                ocr_df=pd.DataFrame(columns=["file", "full_file_path","title",'text','joint_text'])
                ocr_df.to_csv(ocr_mapping_df_path, index=False)
            else:
                ocr_df=pd.read_csv(ocr_mapping_df_path)
    
            for i in range(num_files):    
                while True:  # Loop to retry if an error occurs
                    try:

                        title, text = extract_sanitized(dataset_iterator)

                        joint_text = title + "\n\n" + text
                        

                        creation = super_class_map[super_class][sub_class]
                        path_pdf_file=creation(path_pdf, title, text, i)
                        print(path_pdf_file)

                        file_name=os.path.basename(path_pdf_file)

                        ocr_df.loc[len(ocr_df)] = {
                            "file": file_name,
                            "full_file_path": path_pdf_file,
                            "title": title,
                            "text": text,
                            "joint_text": joint_text
                        }
                        break  # Exit the loop if successful
                    except StopIteration:
                        print("No more elements in the dataset.")
                        break
                    except Exception as e:
                        print(f"Error occurred: {e}. Retrying with the next dataset element..., cout error {count}")
                        print(path_pdf)
                        print (title)
                        print (text)
                        count += 1
                        continue  # Retry with the next dataset element
            ocr_df.to_csv(ocr_mapping_df_path, index=False)

        elif (sub_class=='Font-poisoning_default'):
            font_poisoning_mapping_df_path="../font_poisoning_mapping.csv"
            #if file does not exist, create new df
            if not os.path.exists(font_poisoning_mapping_df_path):
                font_poisoning_df=pd.DataFrame(columns=["file", "full_file_path","title",'text','joint_text'])
                font_poisoning_df.to_csv(font_poisoning_mapping_df_path, index=False)
            else:
                font_poisoning_df=pd.read_csv(font_poisoning_mapping_df_path)
    
            for i in range(num_files):    
                while True:  # Loop to retry if an error occurs
                    try:
                        title, text = extract_sanitized(dataset_iterator)


                        joint_text = title + "\n\n" + text
                        
                        creation = super_class_map[super_class][sub_class]
                        path_pdf_file=creation(path_pdf, title, text, i)
                        print(path_pdf_file)

                        file_name=os.path.basename(path_pdf_file)
                        font_poisoning_df.loc[len(font_poisoning_df)] = {
                            "file": file_name,
                            "full_file_path": path_pdf_file,
                            "title": title,
                            "text": text,
                            "joint_text": joint_text
                        }

                        break  # Exit the loop if successful
                    except StopIteration:
                        print("No more elements in the dataset.")
                        break
                    except Exception as e:
                        print(f"Error occurred: {e}. Retrying with the next dataset element..., cout error {count}")
                        print(path_pdf)
                        print (title)
                        print (text)
                        count += 1
                        continue  # Retry with the next dataset element
            font_poisoning_df.to_csv(font_poisoning_mapping_df_path,index=False)
        
        elif (sub_class ==  "Diacritical-injection_mask1" or sub_class ==  "Diacritical-injection_mask2") :#Camouflage Element
            for i in range(num_files):    
                
                # Iterate until a title with a diacritical accent is found
                found = False
                while not found:
                    try:
                        title, text = extract_sanitized(dataset_iterator)

                        
                        if has_diacritical_accent(title):                       
                            # Create the PDF
                            print(path_pdf)
                            creation = super_class_map[super_class][sub_class]
                            print(creation(path_pdf, title, text, i))
                            found = True
                            
                    except StopIteration:
                        print("No more elements in the dataset.")
                        break  # Exit the loop if no more elements are available
                    except Exception as e:
                        print(f"Error occurred: {e}. Retrying with the next dataset element...")
                        continue  # Retry with the next element if an error occurs

    
        else :
            for i in range(num_files):    
                while True:  # Loop to retry if an error occurs
                    try:
                        title, text = extract_sanitized(dataset_iterator)
                        
                        creation = super_class_map[super_class][sub_class]
                        print(creation(path_pdf, title, text, i))
                        break  # Exit the loop if successful
                    except StopIteration:
                        print("No more elements in the dataset.")
                        break
                    except Exception as e:
                        print(f"Error occurred: {e}. Retrying with the next dataset element..., cout error {count}")
                        print(path_pdf)
                        print (title)
                        print (text)
                        count += 1
                        continue  # Retry with the next dataset element
